In [63]:
%matplotlib inline
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict
import scipy.sparse as sp
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import brier_score_loss

In [64]:
import pymysql 
pymysql.install_as_MySQLdb()

In [65]:
conn_string = 'mysql://{user}:{password}@{host}:{port}/'.format(
    user='root', password='dwdstudent2015', 
    host = '34.225.180.235', port=3306)
engine = create_engine(conn_string, encoding='utf8')

engine.execute('USE ml_football')

/Users/ice/anaconda/lib/python3.5/site-packages/pymysql/cursors.py:165: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [66]:
# all seasons for England
df = pd.read_sql('select Date, HomeTeam, AwayTeam, FTHG - FTAG diff from England', con = engine)

year = []
month = []
results = []
for i in range(df.shape[0]):
    datetime = df['Date'][i].to_pydatetime()
    year.append(datetime.year)
    month.append(datetime.month)
    if df['diff'][i] > 0:
        results.append(1)
    elif df['diff'][i] == 0:
#         results.append(0.5)
        results.append(2)
    else:
        results.append(0)
df['results'] = results
df['year'] = year
df['month'] = month
df.head()

,Date,HomeTeam,AwayTeam,diff,results,year,month
0,2012-08-18,Arsenal,Sunderland,0.0,2,2012,8
1,2012-08-18,Fulham,Norwich,5.0,1,2012,8
2,2012-08-18,Newcastle,Tottenham,1.0,1,2012,8
3,2012-08-18,QPR,Swansea,-5.0,0,2012,8
4,2012-08-18,Reading,Stoke,0.0,2,2012,8


In [67]:
# season 2012 for England
df1 = df.loc[df['year'] == 2012]
df2 = df.loc[df['year'] == 2013].loc[df['month'] < 6]
df_eng12 = pd.concat([df1,df2])
df_eng12.tail()

# df1 = df.loc[df['year'] == 2013].loc[df['month'] > 7]
# df2 = df.loc[df['year'] == 2014].loc[df['month'] < 6]
# df_eng12 = pd.concat([df1,df2])
# df_eng12.head()

,Date,HomeTeam,AwayTeam,diff,results,year,month
375,2013-05-19,Swansea,Fulham,-3.0,0,2013,5
376,2013-05-19,Tottenham,Sunderland,1.0,1,2013,5
377,2013-05-19,West Brom,Man United,0.0,2,2013,5
378,2013-05-19,West Ham,Reading,2.0,1,2013,5
379,2013-05-19,Wigan,Aston Villa,0.0,2,2013,5


In [68]:
# get data
list_file = [(df_eng12['HomeTeam'][i], df_eng12['AwayTeam'][i]) for i in range(len(df_eng12['HomeTeam']))]

v = DictVectorizer()
    
# discover corpus and vectorize file word frequencies in a single pass
X = v.fit_transform(Counter(f) for f in list_file)
# or, if you have a pre-defined corpus and/or would like to restrict the words you consider
# in your matrix, you can do
y1 = df_eng12['diff']
y2 = df_eng12['results']
# Corpus = ('aaa', 'bbb', 'cccc', 'dddd', 'xyz')
# v.fit([OrderedDict.fromkeys(Corpus, 1)])
# X = v.transform(Counter(f) for f in (File_1, File_2))

# X is a sparse matrix, but you can access the A property to get a dense numpy.ndarray 
# representation
#print(X)
#print(X.A[0])

#change to normal matrix so that we can change the values
X = X.A

# this is the dictionary for Teams
print(v.vocabulary_)

for i in range(df_eng12.shape[0]):
    X[i][v.vocabulary_[df_eng12['AwayTeam'][i]]] = -1;

# X is the feature matrix and y is the ground truth vector (for goal difference)

{'Everton': 3, 'West Ham': 18, 'Sunderland': 14, 'Tottenham': 16, 'West Brom': 17, 'Chelsea': 2, 'Man City': 6, 'Liverpool': 5, 'Fulham': 4, 'Arsenal': 0, 'QPR': 10, 'Stoke': 13, 'Aston Villa': 1, 'Swansea': 15, 'Southampton': 12, 'Norwich': 9, 'Reading': 11, 'Man United': 7, 'Newcastle': 8, 'Wigan': 19}


In [76]:
def Brier_Score(true_label, HomeWinProb, DrawProb, AwayWinProb):
    
    """
    @true_label: a list of string to represent the true result, {'HW','Draw','AW'}
    @HomeWinProb:  a list of home wining probability
    @DrawProb: a list of draw probability
    @AwayWinProb: a list of away wining probability
    """
    n = len(true_label)
    
    label_transfer={}
    label_transfer[1]=[1,0,0]
    label_transfer[2]=[0,1,0] 
    label_transfer[0]=[0,0,1] 

    HomeWinProb = np.array(HomeWinProb).reshape((n,1))
    DrawProb = np.array(DrawProb).reshape((n,1))
    AwayWinProb = np.array(AwayWinProb).reshape((n,1))
    true_label = np.array(true_label).reshape((n,1))
    
    total_pro= np.concatenate((HomeWinProb,DrawProb,AwayWinProb),axis=1)
    true_label_matri = [ label_transfer[i[0]] for i in true_label]
    
    true_label_matri = np.asarray(true_label_matri)
    
    res= sum(((true_label_matri-total_pro)**2).T)
    
    return res/3

In [114]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def prob_draw_lose(x):
    if x > 1.5:
        return 1
    elif x < -1.5:
        return 0
    else:
        return float(x) / 3 + 0.5

error_rate_list = []
brier_score_list = []

for j in range(5,0,-1): 
    # hold out the last 5 run as test data
    X_train = X[:-10*j,:]
    X_test = pd.DataFrame(X).tail(10*j)[:10]
    y_train = y1[:-10*j]
    y_train_prob = y2[:-10*j]
    y_test = y1.tail(10*j)[:10]
    y_prob = y2.tail(10*j)[:10] # results: {1,0.5,0}

    # build linear regression model
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    
    reg = linear_model.Ridge(alpha =0.1)
    reg.fit(X_train, y_train)
    y_preds = reg.predict(X_test) # diff
    
    # build multinomial logistic regression
    mr = linear_model.LogisticRegression(tol=1e-10, max_iter=100, solver='sag',multi_class='multinomial')
    mr.fit(X_train,y_train_prob)
    y_preds_mr = mr.predict_proba(X_test)
    y_preds_class = mr.predict(X_test)
    print(mr.coef_)
    error_rate_list.append(round(float(np.mean(y_preds_class != list(y_prob))),4))
    #print(y_preds_mr)
    
    
    
    #print([list(v.vocabulary_.keys())[list(v.vocabulary_.values()).index(i)] for i in sorted(range(len(lr.coef_)), key=lambda k: lr.coef_[k], reverse =True)])
    #print([list(v.vocabulary_.keys())[list(v.vocabulary_.values()).index(i)] for i in sorted(range(len(reg.coef_)), key=lambda k: reg.coef_[k], reverse =True)])
    # turn the diff prediction to three probabilities
    y_probH = []
    y_probD = []
    y_probA = []

    for i in range(len(y_preds)):
        y_probH.append(sigmoid(y_preds[i]))
        y_probD.append(prob_draw_lose(y_preds[i])*(1-y_probH[i]))
        y_probA.append(1 - y_probH[i] - y_probD[i])

    df_prob = pd.DataFrame()
    df_prob['H'] = y_probH
    df_prob['D'] = y_probD
    df_prob['A'] = y_probA
    #print(df_prob)

#     transfer the probabilities to classification problem
#     class_list = [1.0,2.0,0.0]
#     pred_class = [class_list[np.argmax(list(df_prob.iloc[i,:]))] for i in range(df_prob.shape[0])]
#     print(brier_score_loss(y_prob,pred_class))
#     print(np.mean(pred_class == y_prob))
    
    # transfer the predicted probability to specific format
    y_preds_mr_ = np.array([item for sublist in y_preds_mr for item in sublist]).reshape(3,10)
    brier_score_list_mr = []
    brier_score_list_mr.append(Brier_Score(y_prob,y_preds_mr_[0],y_preds_mr_[1],y_preds_mr_[2]))

    brier_score_list.append(round(float(np.mean(brier_score_list_mr)),4))

print(error_rate_list)
print(brier_score_list)
#print(X_test)
#print(y_test)
#print(y_preds)

[[-0.47735877  0.37757448 -0.56009975 -0.29658441  0.39452376 -0.07207658
  -0.70583884 -0.99460452  0.60370488  0.18514881  0.45574218  0.47126577
   0.16482134  0.19284645  0.20787006  0.02475275 -0.60486681  0.12024143
   0.10153601  0.41140179]
 [ 0.48827944 -0.31039078  0.38910674  0.44846307 -0.02796546  0.26485425
   0.83631573  1.17388821 -0.15550414 -0.1762329  -0.74377513 -0.83337369
  -0.19286733 -0.23633812 -0.37398042 -0.13067362  0.27463029 -0.04056467
  -0.21040965 -0.44346181]
 [-0.01092067 -0.0671837   0.17099302 -0.15187865 -0.36655829 -0.19277767
  -0.13047689 -0.17928369 -0.44820074 -0.00891591  0.28803295  0.36210792
   0.02804599  0.04349167  0.16611037  0.10592087  0.33023652 -0.07967676
   0.10887364  0.03206002]]
[[ -4.87019893e-01   3.83176272e-01  -5.25411486e-01  -2.70534697e-01
    4.09782696e-01  -1.64620011e-01  -6.91031742e-01  -1.00998958e+00
    6.84955617e-01   1.86665627e-01   4.59778510e-01   4.75997187e-01
    1.90793962e-01   1.96160150e-01   1.72

/Users/ice/anaconda/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


extract features: home advantage, team ability(+ if it's home team, - if it's away team).